# Find precipitation value for pulsed and upslope storms and create seasonal statistic

In [1]:
import sys
#sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')
sys.path.append('/home/franzihe/Documents/Python/Thesis')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')

import seaborn

In [2]:
m = ['12', '01', '02']
station = 'Haukeliseter'
meps = 'CTRL'
#meps = 'XCCR'

In [3]:
savefig = 0
if savefig == 1:
    figdir = '../../../Figures/Weathermast_MEPS_Retrieval/%s/precip_seasonal_stat' %(station)
    cF.createFolder('%s/' %figdir)
form = 'png'

In [4]:
f = dict()
obs_precip_diff      = dict()
obs_precip_diff_2deg = dict()
obs_west_idx         = dict()
obs_east_idx         = dict()
obs_precip_total     = dict()
obs_precip_west      = dict()
obs_precip_east      = dict()
obs_precip_total_2deg= dict()
obs_precip_west_2deg = dict()
obs_precip_east_2deg = dict()
    
meps_precip_diff     = dict()
meps_precip_diff_2deg= dict()
meps_west_idx        = dict()
meps_east_idx        = dict()
meps_precip_total    = dict()
meps_precip_west     = dict()
meps_precip_east     = dict()
meps_precip_total_2deg= dict()
meps_precip_west_2deg = dict()
meps_precip_east_2deg = dict()
    
ret_precip_diff      = dict()
ret_precip_total     = dict()
ret_precip_west      = dict()
ret_precip_east      = dict()

date = []

for month in m:
    if month == '12' or  month == '01':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    
    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
            

        Date = int(year+month+day)
        nc_dir = '../../../Data/Weathermast_MEPS_Retrieval/%s/%s/weathermast_meps_retrieval_%s_12.nc' %(station, meps, Date)

        f[Date] = netCDF4.Dataset(nc_dir, 'r')
        if len(f[Date].variables) == 0:
            print(Date, 'no valid data')
        else:
            if np.nansum(f[Date].variables['obs_precipitation_difference_2degC']) >= 0.25 and np.nansum(f[Date].variables['meps_precipitation_difference_2degC']) >= 0.25 and np.nansum(f[Date].variables['ret_precipitation_difference']) >= 0.25:
                #print(Date)
                date = np.append(date, int(Date))

                obs_precip_diff[Date]      = f[Date].variables['obs_precipitation_difference'][:]
                obs_precip_diff_2deg[Date] = f[Date].variables['obs_precipitation_difference_2degC'][:]
                obs_west_idx[Date]         = f[Date].variables['obs_idx_westerly'][:].astype(int)
                obs_east_idx[Date]         = f[Date].variables['obs_idx_easterly'][:].astype(int)

                meps_precip_diff[Date]     = f[Date].variables['meps_precipitation_difference'][:]
                meps_precip_diff_2deg[Date]= f[Date].variables['meps_precipitation_difference_2degC'][:]
                meps_west_idx[Date]        = f[Date].variables['meps_idx_westerly'][:]
                meps_east_idx[Date]        = f[Date].variables['meps_idx_easterly'][:]

                ret_precip_diff[Date]      = f[Date].variables['ret_precipitation_difference'][:]



                ############################################
                if len(obs_west_idx[Date]) == 0 and len(obs_east_idx[Date]) == 0:
                    obs_precip_total[Date] = np.nan
                    obs_precip_west[Date]  = np.nan
                    obs_precip_east[Date]  = np.nan
                ### observation == reference
                obs_precip_total[Date] = np.nansum(obs_precip_diff[Date][obs_west_idx[Date],]) + \
                                         np.nansum(obs_precip_diff[Date][obs_east_idx[Date],])
                obs_precip_west[Date]  = np.nansum(obs_precip_diff[Date][obs_west_idx[Date],])
                obs_precip_east[Date]  = np.nansum(obs_precip_diff[Date][obs_east_idx[Date],])
                ### retrieval
                ret_precip_total[Date] = np.nansum(ret_precip_diff[Date][obs_west_idx[Date],]) + \
                                         np.nansum(ret_precip_diff[Date][obs_east_idx[Date],])
                ret_precip_west[Date]  = np.nansum(ret_precip_diff[Date][obs_west_idx[Date],])
                ret_precip_east[Date]  = np.nansum(ret_precip_diff[Date][obs_east_idx[Date],])
                ### MEPS
        #        meps_precip_total[Date] = []
         #       meps_precip_west[Date]  = []
          #      meps_precip_east[Date]  = []
           #     for ens_memb in range(10):
        #            total = np.nansum(meps_precip_diff[Date][np.where(~np.isnan(meps_west_idx[Date][:,ens_memb])), ens_memb]) + \
         #                   np.nansum(meps_precip_diff[Date][np.where(~np.isnan(meps_east_idx[Date][:,ens_memb])), ens_memb])
          #          meps_precip_total[Date].append(total)  
                meps_precip_total[Date] = np.nansum(meps_precip_diff[Date][~np.isnan(meps_west_idx[Date]),]) + \
                                          np.nansum(meps_precip_diff[Date][~np.isnan(meps_east_idx[Date]),]) 

        #            westerly = np.nansum(meps_precip_diff[Date][np.where(~np.isnan(meps_west_idx[Date][:,ens_memb])), ens_memb])
         #           meps_precip_west[Date].append(westerly)
                meps_precip_west[Date] =  np.nansum(meps_precip_diff[Date][~np.isnan(meps_west_idx[Date]),])

        #            easterly = np.nansum(meps_precip_diff[Date][np.where(~np.isnan(meps_east_idx[Date][:,ens_memb])), ens_memb])
         #           meps_precip_east[Date].append(easterly) 
                meps_precip_east[Date] =  np.nansum(meps_precip_diff[Date][~np.isnan(meps_east_idx[Date]),])

          #          del total, westerly, easterly

                ## for precipitation below 2degC
                ### observation
                obs_precip_total_2deg[Date] = np.nansum(obs_precip_diff_2deg[Date][obs_west_idx[Date],]) + \
                                              np.nansum(obs_precip_diff_2deg[Date][obs_east_idx[Date],])
                obs_precip_west_2deg[Date]  = np.nansum(obs_precip_diff_2deg[Date][obs_west_idx[Date],])
                obs_precip_east_2deg[Date]  = np.nansum(obs_precip_diff_2deg[Date][obs_east_idx[Date],])
                ### MEPS
        #        meps_precip_total_2deg[Date] = []
         #       meps_precip_west_2deg[Date]  = []
          #      meps_precip_east_2deg[Date]  = []
        #        for ens_memb in range(10):
         #           total = np.nansum(meps_precip_diff_2deg[Date][np.where(~np.isnan(meps_west_idx[Date][:,ens_memb])), ens_memb]) + \
          #                  np.nansum(meps_precip_diff_2deg[Date][np.where(~np.isnan(meps_east_idx[Date][:,ens_memb])), ens_memb])
           #         meps_precip_total_2deg[Date].append(total)
                meps_precip_total_2deg[Date] = np.nansum(meps_precip_diff_2deg[Date][~np.isnan(meps_west_idx[Date]),]) + \
                                               np.nansum(meps_precip_diff_2deg[Date][~np.isnan(meps_east_idx[Date]),])

        #            westerly = np.nansum(meps_precip_diff_2deg[Date][np.where(~np.isnan(meps_west_idx[Date][:,ens_memb])), ens_memb])
         #           meps_precip_west_2deg[Date].append(westerly)            
                meps_precip_west_2deg[Date] = np.nansum(meps_precip_diff_2deg[Date][~np.isnan(meps_west_idx[Date]),])

        #            easterly = np.nansum(meps_precip_diff_2deg[Date][np.where(~np.isnan(meps_east_idx[Date][:,ens_memb])), ens_memb])
         #           meps_precip_east_2deg[Date].append(easterly)            
                meps_precip_east_2deg[Date] = np.nansum(meps_precip_diff_2deg[Date][~np.isnan(meps_east_idx[Date]),])


                print(int(Date), np.round(obs_precip_west_2deg[Date],2),
                                 np.round(obs_precip_east_2deg[Date],2),
                                 np.round(obs_precip_total_2deg[Date],2))
        f[Date].close()
############################

20161201 2.1 0.0 2.1
20161206 5.69 1.99 7.68
20161208 3.98 0.0 3.98
20161209 4.93 2.15 7.08
20161210 1.8 0.0 1.8
20161212 0.73 0.12 0.85
20161215 0.15 0.22 0.37
20161216 no valid data
20161217 no valid data
20161218 no valid data
20161220 0.0 2.65 2.65
20161221 27.72 0.59 28.31
20161222 15.39 0.19 15.58
20161223 17.86 14.54 32.39
20161224 10.22 0.0 10.22
20161225 11.4 0.0 11.4
20161226 14.67 0.0 14.67
20161229 1.4 0.0 1.4
20161231 2.45 0.0 2.45
20170102 3.66 0.05 3.71
20170103 3.08 0.0 3.08
20170105 0.21 0.48 0.69
20170106 0.34 1.51 1.85
20170108 0.6 0.3 0.9
20170109 0.0 4.77 4.77
20170110 5.5 9.71 15.21
20170111 16.33 0.0 16.33
20170112 6.89 1.77 8.66
20170116 1.6 0.0 1.6
20170128 0.44 3.04 3.49
20170129 no valid data
20170130 no valid data
20170131 no valid data
20170202 0.0 3.7 3.7
20170203 0.39 6.4 6.79
20170204 0.02 3.44 3.46
20170206 0.0 2.8 2.8
20170207 no valid data
20170208 no valid data
20170209 no valid data
20170210 no valid data
20170211 no valid data
20170212 no valid dat

In [5]:
# calculate monthly precip sum observation for <2degC, and valid days
obs_total_2deg = []
obs_total_east_2deg = []
obs_total_west_2deg = []


obs_monthly_sum_2deg = 0.0
obs_monthly_east_sum_2deg = 0.0
obs_monthly_west_sum_2deg = 0.0
for Date in date:
    Date = int(Date)
    
    if Date == int(20161201) or Date == int(20170102) or Date == int(20170202):
        obs_monthly_sum_2deg = 0.0
        obs_monthly_east_sum_2deg = 0.0
        obs_monthly_west_sum_2deg = 0.0
    
    tot_val_2deg = (obs_precip_total_2deg[Date])
    obs_monthly_sum_2deg = np.sum([obs_monthly_sum_2deg,tot_val_2deg])

    east_val_2deg = (obs_precip_east_2deg[Date])
    obs_monthly_east_sum_2deg = np.sum([obs_monthly_east_sum_2deg, east_val_2deg])

    west_val_2deg = (obs_precip_west_2deg[Date])
    obs_monthly_west_sum_2deg = np.sum([obs_monthly_west_sum_2deg, west_val_2deg])

    if Date == int(20161231) or Date == int(20170128) or Date == int(20170206):
                obs_total_2deg.append(obs_monthly_sum_2deg)
                obs_total_east_2deg.append(obs_monthly_east_sum_2deg)
                obs_total_west_2deg.append(obs_monthly_west_sum_2deg)

    print(Date,'west:',  np.round(obs_monthly_west_sum_2deg,2),
               'east:',  np.round(obs_monthly_east_sum_2deg,2),
               'total:', np.round(obs_monthly_sum_2deg,2))
    
#print('total:', np.round(sum(obs_total_precip.values()),2), 
 #     'east:', np.round(sum(obs_precip_east.values()),2), 
  #    'west:', np.round(sum(obs_precip_west.values()),2))

print('west:',  np.round(np.sum(obs_total_west_2deg), 2),
      'east:',  np.round(np.sum(obs_total_east_2deg), 2),
      'total:', np.round(np.sum(obs_total_2deg),2),)

20161201 west: 2.1 east: 0.0 total: 2.1
20161206 west: 7.79 east: 1.99 total: 9.78
20161208 west: 11.77 east: 1.99 total: 13.76
20161209 west: 16.7 east: 4.13 total: 20.83
20161210 west: 18.5 east: 4.13 total: 22.64
20161212 west: 19.24 east: 4.26 total: 23.49
20161215 west: 19.39 east: 4.48 total: 23.87
20161220 west: 19.39 east: 7.13 total: 26.52
20161221 west: 47.11 east: 7.72 total: 54.83
20161222 west: 62.5 east: 7.91 total: 70.41
20161223 west: 80.36 east: 22.44 total: 102.8
20161224 west: 90.58 east: 22.44 total: 113.03
20161225 west: 101.98 east: 22.44 total: 124.42
20161226 west: 116.65 east: 22.44 total: 139.09
20161229 west: 118.05 east: 22.44 total: 140.49
20161231 west: 120.5 east: 22.44 total: 142.94
20170102 west: 3.66 east: 0.05 total: 3.71
20170103 west: 6.74 east: 0.05 total: 6.79
20170105 west: 6.95 east: 0.53 total: 7.48
20170106 west: 7.29 east: 2.04 total: 9.33
20170108 west: 7.89 east: 2.34 total: 10.23
20170109 west: 7.89 east: 7.11 total: 14.99
20170110 west: 1

### RETRIEVAL

In [6]:
### MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [7]:
# calculate monthly precip sum for retrieved precipitation
ret_total = []
ret_total_east = []
ret_total_west = []

ret_monthly_sum = 0.0
ret_monthly_east_sum = 0.0
ret_monthly_west_sum = 0.0

for Date in date:
    Date = int(Date)
    
    if Date == int(20161201) or Date == int(20170102) or Date == int(20170202):
        ret_monthly_sum = 0.0
        ret_monthly_east_sum = 0.0
        ret_monthly_west_sum = 0.0

    tot_val = (ret_precip_total[Date])
    ret_monthly_sum = np.sum([ret_monthly_sum,tot_val])
        
    east_val = (ret_precip_east[Date])
    ret_monthly_east_sum = np.sum([ret_monthly_east_sum, east_val])
        
    west_val = (ret_precip_west[Date])
    ret_monthly_west_sum = np.sum([ret_monthly_west_sum, west_val])
        
    if Date == int(20161231) or Date == int(20170128) or Date == int(20170206):
            ret_total.append(ret_monthly_sum)
            ret_total_east.append(ret_monthly_east_sum)
            ret_total_west.append(ret_monthly_west_sum)
            
    print(Date,'west:',  np.round(ret_monthly_west_sum,2),
               'east:',  np.round(ret_monthly_east_sum,2),
               'total:', np.round(ret_monthly_sum,2))
#print('total:', np.round(sum(ret_precip_total.values()),2), 
 #     'east:', np.round(sum(ret_precip_east.values()),2), 
  #    'west:', np.round(sum(ret_precip_west.values()),2))

ret_sum = np.sum(ret_total,axis=0)
ret_east_sum = np.sum(ret_total_east, axis=0)
ret_west_sum = np.sum(ret_total_west, axis=0)

print('west:',  np.round(np.sum(ret_total_west), 2),
      'east:',  np.round(np.sum(ret_total_east), 2),
      'total:', np.round(np.sum(ret_total),2),)

20161201 west: 0.27 east: 0.0 total: 0.27
20161206 west: 5.68 east: 2.06 total: 7.73
20161208 west: 6.78 east: 2.06 total: 8.83
20161209 west: 7.0 east: 3.78 total: 10.77
20161210 west: 7.38 east: 3.78 total: 11.16
20161212 west: 7.97 east: 3.86 total: 11.83
20161215 west: 8.43 east: 3.94 total: 12.38
20161220 west: 8.43 east: 8.78 total: 17.22
20161221 west: 39.85 east: 9.5 total: 49.35
20161222 west: 61.0 east: 9.61 total: 70.61
20161223 west: 83.33 east: 22.84 total: 106.18
20161224 west: 100.1 east: 22.84 total: 122.94
20161225 west: 115.97 east: 22.84 total: 138.82
20161226 west: 123.7 east: 22.84 total: 146.54
20161229 west: 124.53 east: 22.84 total: 147.38
20161231 west: 127.28 east: 22.84 total: 150.12
20170102 west: 4.58 east: 0.04 total: 4.61
20170103 west: 5.81 east: 0.04 total: 5.84
20170105 west: 6.15 east: 0.76 total: 6.91
20170106 west: 6.38 east: 2.83 total: 9.2
20170108 west: 6.87 east: 3.09 total: 9.96
20170109 west: 6.87 east: 15.26 total: 22.13
20170110 west: 14.15 

In [8]:
### retrieval seasonal statistics
diff_perc_monthly = np.zeros(shape=np.array(ret_total).shape)
diff_perc_season = np.zeros(shape=ret_sum.shape)

diff_perc_monthly_east = np.zeros(shape=np.array(ret_total_east).shape)
diff_perc_season_east = np.zeros(shape=ret_east_sum.shape)

diff_perc_monthly_west = np.zeros(shape=np.array(ret_total_west).shape)
diff_perc_season_west = np.zeros(shape=ret_west_sum.shape)

diff_perc_monthly = (ret_total - np.array(obs_total_2deg))/np.array(obs_total_2deg)*100
diff_perc_season  = (np.sum(ret_total) - np.sum(obs_total_2deg))/np.sum(obs_total_2deg)*100
    
diff_perc_monthly_east = (ret_total_east - np.array(obs_total_east_2deg))/np.array(obs_total_east_2deg)*100
diff_perc_season_east  = (ret_east_sum - np.sum(obs_total_east_2deg))/np.sum(obs_total_east_2deg)*100
    
diff_perc_monthly_west = (ret_total_west - np.array(obs_total_west_2deg))/np.array(obs_total_west_2deg)*100
diff_perc_season_west = (ret_west_sum - np.sum(obs_total_west_2deg))/np.sum(obs_total_west_2deg)*100

### create table

col_labels = ['obs [mm]', 'retrieved [mm]','Difference [%]', ]
row_labels = ['total precipitation', 'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '', 
              'pulsed - westerly',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '',
              'upslope - easterly',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017']
#for  in range(10):

table_vals = [ ['', '' , '' ],
               [np.round(obs_total_2deg[0],2), np.round(ret_total[0],2), np.round(diff_perc_monthly[0],2)],
               [np.round(obs_total_2deg[1],2), np.round(ret_total[1],2), np.round(diff_perc_monthly[1],2)],
               [np.round(obs_total_2deg[2],2), np.round(ret_total[2],2), np.round(diff_perc_monthly[2],2)],
#               [np.round(obs_total_2deg[3],2), np.round(ret_total[3],2), np.round(diff_perc_monthly[3],2)],
 #              [np.round(obs_total_2deg[4],2), np.round(ret_total[4],2), np.round(diff_perc_monthly[4],2)],
               [np.round(np.sum(obs_total_2deg),2), np.round(ret_sum,2), np.round(diff_perc_season,2)],
               ['', '', ''],
               ['', '', '' ],
               [np.round(obs_total_west_2deg[0],2), np.round(ret_total_west[0],2), np.round(diff_perc_monthly_west[0],2)],
               [np.round(obs_total_west_2deg[1],2), np.round(ret_total_west[1],2), np.round(diff_perc_monthly_west[1],2)],
               [np.round(obs_total_west_2deg[2],2), np.round(ret_total_west[2],2), np.round(diff_perc_monthly_west[2],2)],
#               [np.round(obs_total_west_2deg[3],2), np.round(ret_total_west[3],2), np.round(diff_perc_monthly_west[3],2)],
 #              [np.round(obs_total_west_2deg[4],2), np.round(ret_total_west[4],2), np.round(diff_perc_monthly_west[4],2)],
               [np.round(np.sum(obs_total_west_2deg),2), np.round(ret_west_sum,2), np.round(diff_perc_season_west,2)],
               ['', '', ''],
               ['', '', '' ],
               [np.round(obs_total_east_2deg[0],2), np.round(ret_total_east[0],2), np.round(diff_perc_monthly_east[0],2)],
               [np.round(obs_total_east_2deg[1],2), np.round(ret_total_east[1],2), np.round(diff_perc_monthly_east[1],2)],
               [np.round(obs_total_east_2deg[2],2), np.round(ret_total_east[2],2), np.round(diff_perc_monthly_east[2],2)],
#               [np.round(obs_total_east_2deg[3],2), np.round(ret_total_east[3],2), np.round(diff_perc_monthly_east[3],2)],
 #              [np.round(obs_total_east_2deg[4],2), np.round(ret_total_east[4],2), np.round(diff_perc_monthly_east[4],2)],
               [np.round(np.sum(obs_total_east_2deg),2), np.round(ret_east_sum,2), np.round(diff_perc_season_east,2)]]


# Draw table
fig = plt.figure()
ax = fig.add_subplot(111)
the_table = plt.table(cellText = table_vals,
                     rowLabels = row_labels,
                     colLabels = col_labels,
                     loc = 'center'
                     )
the_table.auto_set_font_size(False)
the_table.set_fontsize(24)
the_table.scale(3,3.5)

# Removing ticks and spines enables you to get the figure only with table
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    
yy = 2.33
fig_name = 'winter_16_17_retrieval_2degC'+'.'+form
ax.set_title( 'Winter 2016-2017 -- 24h accumulation', y =yy,
                 fontsize = 24, fontweight='bold')

if savefig == 1:
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/%s' %(figdir,fig_name))
else:
        plt.show()
plt.close()
    
    

plot saved: ../../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/precip_seasonal_stat/winter_16_17_retrieval_2degC.png


### MEPS

In [9]:
# calculate monthly precip sum observation for <2degC, and valid days

meps_sum_2deg = 0.0
meps_east_sum_2deg = 0.0
meps_west_sum_2deg = 0.0
meps_total_2deg = []
meps_total_east_2deg = []
meps_total_west_2deg = []

for Date in date:
    Date = int(Date)
    
    if Date == int(20161201) or Date == int(20170102) or Date == int(20170202):
        meps_monthly_sum_2deg = 0.0
        meps_monthly_east_sum_2deg = 0.0
        meps_monthly_west_sum_2deg = 0.0
        
    tot_val_2deg = meps_precip_total_2deg[Date]
    meps_monthly_sum_2deg = np.nansum([meps_monthly_sum_2deg, tot_val_2deg])
        
    east_val_2deg = meps_precip_east_2deg[Date]
    meps_monthly_east_sum_2deg = np.nansum([meps_monthly_east_sum_2deg, east_val_2deg])
        
    west_val_2deg = meps_precip_west_2deg[Date]
    meps_monthly_west_sum_2deg = np.nansum([meps_monthly_west_sum_2deg, west_val_2deg])
        
    if Date == int(20161231) or Date == int(20170128) or Date == int(20170206):
                meps_total_2deg.append(meps_monthly_sum_2deg)
                meps_total_east_2deg.append(meps_monthly_east_sum_2deg)
                meps_total_west_2deg.append(meps_monthly_west_sum_2deg)

    print(Date, '%s' %meps, 
                               'west:',  np.round(meps_monthly_west_sum_2deg,2),
                               'east:',  np.round(meps_monthly_east_sum_2deg,2),
                               'total:', np.round(meps_monthly_sum_2deg,2))


#meps_total_2deg = (np.reshape(meps_total_2deg, (5,10)))
#meps_total_east_2deg = (np.reshape(meps_total_east_2deg, (5,10)))
#meps_total_west_2deg = (np.reshape(meps_total_west_2deg, (5,10)))

meps_sum_2deg = np.sum(meps_total_2deg,axis=0)
meps_east_sum_2deg = np.sum(meps_total_east_2deg, axis=0)
meps_west_sum_2deg = np.sum(meps_total_west_2deg, axis=0)

#for ens_memb in range(10):
print('total:', '%s' %meps, 'west:', np.round(meps_west_sum_2deg,2),
                            'east:', np.round(meps_east_sum_2deg,2),
                            'total:', np.round(meps_sum_2deg,2))


20161201 CTRL west: 0.37 east: 0.0 total: 0.37
20161206 CTRL west: 0.8 east: 6.24 total: 7.04
20161208 CTRL west: 7.91 east: 6.24 total: 14.15
20161209 CTRL west: 11.89 east: 9.19 total: 21.07
20161210 CTRL west: 18.31 east: 9.19 total: 27.5
20161212 CTRL west: 19.08 east: 9.5 total: 28.58
20161215 CTRL west: 19.08 east: 9.82 total: 28.9
20161220 CTRL west: 21.04 east: 14.58 total: 35.62
20161221 CTRL west: 40.63 east: 14.58 total: 55.21
20161222 CTRL west: 74.99 east: 14.58 total: 89.57
20161223 CTRL west: 107.37 east: 22.3 total: 129.67
20161224 CTRL west: 129.09 east: 22.3 total: 151.39
20161225 CTRL west: 153.62 east: 22.3 total: 175.92
20161226 CTRL west: 182.12 east: 22.3 total: 204.42
20161229 CTRL west: 185.83 east: 22.3 total: 208.13
20161231 CTRL west: 189.27 east: 22.3 total: 211.57
20170102 CTRL west: 4.62 east: 0.0 total: 4.62
20170103 CTRL west: 12.41 east: 0.0 total: 12.41
20170105 CTRL west: 13.34 east: 0.0 total: 13.34
20170106 CTRL west: 15.88 east: 0.0 total: 15.88
2

In [10]:
diff_perc_monthly = np.zeros(shape=np.array(meps_total_2deg).shape)
diff_perc_season = np.zeros(shape=meps_sum_2deg.shape)

diff_perc_monthly_east = np.zeros(shape=np.array(meps_total_east_2deg).shape)
diff_perc_season_east = np.zeros(shape=meps_east_sum_2deg.shape)

diff_perc_monthly_west = np.zeros(shape=np.array(meps_total_west_2deg).shape)
diff_perc_season_west = np.zeros(shape=meps_west_sum_2deg.shape)

diff_perc_monthly = (meps_total_2deg - np.array(obs_total_2deg))/np.array(obs_total_2deg)*100
diff_perc_season  = (meps_sum_2deg - np.sum(obs_total_2deg))/np.sum(obs_total_2deg)*100
    
diff_perc_monthly_east = (meps_total_east_2deg - np.array(obs_total_east_2deg))/np.array(obs_total_east_2deg)*100
diff_perc_season_east  = (meps_east_sum_2deg - np.sum(obs_total_east_2deg))/np.sum(obs_total_east_2deg)*100
    
diff_perc_monthly_west = (meps_total_west_2deg - np.array(obs_total_west_2deg))/np.array(obs_total_west_2deg)*100
diff_perc_season_west  = (meps_west_sum_2deg - np.sum(obs_total_west_2deg))/np.sum(obs_total_west_2deg)*100

In [11]:
    ### create table

    col_labels = ['obs [mm]', '%s [mm]' %meps,'Difference [%]', ]
#row_labels = ['total precipitation', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
 #             '', 
  #            'pulsed - westerly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017',
   #           '',
    #          'upslope - easterly', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017', 'Mar 2017', 'Winter 2016-2017']
#for ens_memb in range(10):
    table_vals = [ ['', '', '' ],
               [np.round(obs_total_2deg[0],2), np.round(meps_total_2deg[0],2), np.round(diff_perc_monthly[0],2)],
               [np.round(obs_total_2deg[1],2), np.round(meps_total_2deg[1],2), np.round(diff_perc_monthly[1],2)],
               [np.round(obs_total_2deg[2],2), np.round(meps_total_2deg[2],2), np.round(diff_perc_monthly[2],2)],
#               [np.round(obs_total_2deg[3],2), np.round(meps_total_2deg[3,ens_memb],2), np.round(diff_perc_monthly[3,ens_memb],2)],
 #              [np.round(obs_total_2deg[4],2), np.round(meps_total_2deg[4,ens_memb],2), np.round(diff_perc_monthly[4,ens_memb],2)],
               [np.round(np.sum(obs_total_2deg),2), np.round(meps_sum_2deg,2), np.round(diff_perc_season,2)],
               ['', '', ''],
               ['', '', '' ],
               [np.round(obs_total_west_2deg[0],2), np.round(meps_total_west_2deg[0],2), np.round(diff_perc_monthly_west[0],2)],
               [np.round(obs_total_west_2deg[1],2), np.round(meps_total_west_2deg[1],2), np.round(diff_perc_monthly_west[1],2)],
               [np.round(obs_total_west_2deg[2],2), np.round(meps_total_west_2deg[2],2), np.round(diff_perc_monthly_west[2],2)],
#               [np.round(obs_total_west_2deg[3],2), np.round(meps_total_west_2deg[3,ens_memb],2), np.round(diff_perc_monthly_west[3,ens_memb],2)],
 #              [np.round(obs_total_west_2deg[4],2), np.round(meps_total_west_2deg[4,ens_memb],2), np.round(diff_perc_monthly_west[4,ens_memb],2)],
               [np.round(np.sum(obs_total_west_2deg),2), np.round(meps_west_sum_2deg,2), np.round(diff_perc_season_west,2)],
               ['', '', ''],
               ['', '', ''],
               [np.round(obs_total_east_2deg[0],2), np.round(meps_total_east_2deg[0],2), np.round(diff_perc_monthly_east[0],2)],
               [np.round(obs_total_east_2deg[1],2), np.round(meps_total_east_2deg[1],2), np.round(diff_perc_monthly_east[1],2)],
               [np.round(obs_total_east_2deg[2],2), np.round(meps_total_east_2deg[2],2), np.round(diff_perc_monthly_east[2],2)],
#               [np.round(obs_total_east_2deg[3],2), np.round(meps_total_east_2deg[3,ens_memb],2), np.round(diff_perc_monthly_east[3,ens_memb],2)],
 #              [np.round(obs_total_east_2deg[4],2), np.round(meps_total_east_2deg[4,ens_memb],2), np.round(diff_perc_monthly_east[4,ens_memb],2)],
               [np.round(np.sum(obs_total_east_2deg),2), np.round(meps_east_sum_2deg,2), np.round(diff_perc_season_east,2)]]


# Draw table
    fig = plt.figure()
    ax = fig.add_subplot(111)
    the_table = plt.table(cellText = table_vals,
                     rowLabels = row_labels,
                     colLabels = col_labels,
                     loc = 'center'
                     )
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(24)
    the_table.scale(3,3.5)

# Removing ticks and spines enables you to get the figure only with table
    plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
    for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
    
    yy = 2.33
    fig_name = 'winter_16_17_'+str(meps)+'_2degC.'+form
    ax.set_title( 'Winter 2016-2017 -- 24h accumulation', y =yy,
                 fontsize = 24, fontweight='bold')

    if savefig == 1:
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/%s' %(figdir,fig_name))
    else:
        plt.show()
    plt.close()
    

plot saved: ../../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/precip_seasonal_stat/winter_16_17_CTRL_2degC.png
